In [1]:
import cv2
from ultralytics import YOLO
from IPython.display import display, Image # 用于在 Notebook 中显示静态图像（可选）
import time # 用于计算 FPS (可选)

print("库导入完成。")

库导入完成。


In [2]:
try:
    model = YOLO('yolov8n.pt')
    print("YOLOv8n 模型加载成功。")
except Exception as e:
    print(f"加载 YOLO 模型失败: {e}")
    # 如果模型加载失败，后续单元格可能无法正常工作
    # 考虑在这里停止或给出更明确的指示


YOLOv8n 模型加载成功。


In [13]:
# 尝试打开默认摄像头 (通常索引为 0)
cap = cv2.VideoCapture(0) # 0 代表默认摄像头，如果有多个摄像头，可以尝试 1, 2, ...

if not cap.isOpened():
    print("错误：无法打开摄像头。请检查摄像头是否连接或被其他程序占用。")
    # 如果摄像头打开失败，后续单元格将无法执行
else:
    print("摄像头已成功打开。准备进行实时检测。")
    print("在检测窗口激活时，按 'q' 键退出。")


摄像头已成功打开。准备进行实时检测。
在检测窗口激活时，按 'q' 键退出。


In [14]:
if 'cap' in locals() and cap.isOpened() and 'model' in locals():
    # (可选) 用于计算 FPS
    fps_update_interval = 1 # 更新 FPS 显示的秒数
    frame_count = 0
    start_time = time.time()

    try:
        while True:
            # 从摄像头读取一帧
            ret, frame = cap.read()
            if not ret:
                print("错误：无法从摄像头读取帧。")
                break

            # 使用 YOLOv8n 进行目标检测
            results = model(frame, verbose=False) # verbose=False 减少控制台输出
            annotated_frame = results[0].plot()

            # (可选) 计算并显示 FPS
            current_time = time.time()
            frame_count += 1
            elapsed_time = current_time - start_time

            if elapsed_time >= fps_update_interval:
                fps = frame_count / elapsed_time
                cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (10, 30),
                            cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
                # 重置计数器
                frame_count = 0
                start_time = current_time
            elif frame_count > 0 : # 如果还没到更新间隔，但已有帧，也显示上一次的FPS
                 # 为了避免除以0，确保elapsed_time > 0
                if elapsed_time > 0:
                    fps = frame_count / elapsed_time
                    cv2.putText(annotated_frame, f"FPS: {fps:.2f}", (10, 30),
                                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)


            # 显示带有检测结果的帧
            cv2.imshow('YOLOv8 实时检测 (按 q 退出)', annotated_frame)

            # 检测按键，如果按下 'q' 键，则退出循环
            if cv2.waitKey(1) & 0xFF == ord('q'):
                print("检测已停止。")
                break

    except KeyboardInterrupt:
        print("检测被用户中断 (Ctrl+C)。")
    except Exception as e:
        print(f"实时检测过程中发生错误: {e}")
    finally:
        # 释放摄像头资源
        if cap.isOpened():
            cap.release()
            print("摄像头已释放。")
        # 关闭所有 OpenCV 创建的窗口
        cv2.destroyAllWindows()
        print("OpenCV 窗口已关闭。")
        # 在Jupyter中，有时需要手动关闭弹窗或重启内核以确保窗口完全消失
else:
    if 'cap' not in locals() or not cap.isOpened():
        print("摄像头未初始化或打开失败，请先运行前面的单元格。")
    if 'model' not in locals():
        print("YOLO 模型未加载，请先运行模型加载单元格。")
    # 如果在运行主循环后，窗口仍然存在，可以尝试运行此单元格
    # 或者在中断内核后，运行此单元格来尝试清理
    if 'cap' in locals() and cap.isOpened():
        cap.release()
        print("摄像头已手动释放。")
    cv2.destroyAllWindows()
    print("所有 OpenCV 窗口已尝试关闭。")
    print("提示：如果窗口仍然存在，您可能需要重启 Jupyter 内核。")

    






检测被用户中断 (Ctrl+C)。
摄像头已释放。
OpenCV 窗口已关闭。
